# VectrixDB LangChain Integration

Using VectrixDB with LangChain for RAG applications.

Note: This shows a simple custom integration pattern.

## Setup

First, set up your OpenAI API key for LangChain.

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Check if API key is set
if not os.getenv("OPENAI_API_KEY"):
    print("Warning: OPENAI_API_KEY not set. Create a .env file with your key.")
    print("Example: OPENAI_API_KEY=sk-...")
else:
    print("OpenAI API key loaded.")

OpenAI API key loaded.


## Basic LangChain Integration

In [2]:
from vectrixdb import Vectrix

# Create VectrixDB instance
db = Vectrix("langchain_test", tier="hybrid", language="en")

# Add documents
db.add([
    "VectrixDB is a lightweight vector database.",
    "It supports hybrid search with dense and sparse vectors.",
    "LangChain integration enables RAG applications.",
    "The database runs entirely locally without external services."
])

print(f"Database ready with {len(db)} documents.")

Database ready with 4 documents.


## Simple Retriever Pattern

You can easily create a retriever function for use with LangChain.

In [3]:
# Create a simple retriever function
def retrieve(query: str, k: int = 3):
    """Retrieve relevant documents from VectrixDB."""
    results = db.search(query, limit=k)
    return [r.text for r in results]

# Test retriever
docs = retrieve("vector search")
print("Retrieved documents:")
for doc in docs:
    print(f"  - {doc}")

Retrieved documents:
  - It supports hybrid search with dense and sparse vectors.
  - VectrixDB is a lightweight vector database.
  - LangChain integration enables RAG applications.


## Search with Scores

In [4]:
# Get results with scores
results = db.search("local database", limit=3)

print("Search results with scores:")
for r in results:
    print(f"  Score: {r.score:.4f} | {r.text}")

Search results with scores:
  Score: 0.0180 | The database runs entirely locally without external services.
  Score: 0.0177 | VectrixDB is a lightweight vector database.
  Score: 0.0079 | It supports hybrid search with dense and sparse vectors.


## LangChain Document Format

In [5]:
# Convert VectrixDB results to LangChain Document format
try:
    from langchain.schema import Document
    
    def search_as_documents(query: str, k: int = 3):
        """Search and return LangChain Documents."""
        results = db.search(query, limit=k)
        return [
            Document(
                page_content=r.text,
                metadata=r.metadata or {}
            )
            for r in results
        ]
    
    # Test
    lc_docs = search_as_documents("hybrid search")
    print("LangChain Documents:")
    for doc in lc_docs:
        print(f"  - {doc.page_content}")
except ImportError:
    print("LangChain not installed. Install with: pip install langchain")

LangChain not installed. Install with: pip install langchain


## Adding Documents with Metadata

In [6]:
# Add documents with metadata
db.add(
    texts=[
        "VectrixDB supports multiple search modes.",
        "Reranking improves search quality."
    ],
    metadata=[
        {"source": "docs", "page": 1},
        {"source": "docs", "page": 2}
    ]
)

print(f"Total documents now: {len(db)}")

Total documents now: 6


## Search with Metadata Filter

In [7]:
# Search with metadata filter
results = db.search("search modes", filter={"source": "docs"}, limit=3)

print("Filtered search results (source=docs):")
for r in results:
    print(f"  - {r.text}")
    print(f"    Metadata: {r.metadata}")

Filtered search results (source=docs):
  - VectrixDB supports multiple search modes.
    Metadata: {'source': 'docs', 'page': 1}
  - Reranking improves search quality.
    Metadata: {'source': 'docs', 'page': 2}


## Cleanup

In [8]:
import shutil, os

if os.path.exists("langchain_test"):
    shutil.rmtree("langchain_test")
    print("Deleted langchain_test")